In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.autograd import Variable
from torch import nn

In [2]:
# cv2.circle?
# nn.Conv2d?
# nn.MaxPool2d?
# nn.Linear?
# nn.MSELoss?

line(img, pt1, pt2, color[, thickness[, lineType[, shift]]])

circle(img, center, radius, color[, thickness[, lineType[, shift]]])

nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)

nn.MaxPool2d(self, kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

nn.Linear(self, in_features, out_features, bias=True)

In [3]:
# height, width = 96, 96

In [4]:
training_df = pd.read_csv('../data/training.csv')

In [5]:
# 去除缺失项
training_df = training_df.dropna()

In [6]:
def str2array(data_str):
    data_array = np.fromstring(data_str, dtype=np.float32, sep=' ')
    return data_array

In [7]:
y = training_df.values[:, :-1]
y = np.array(y, dtype=np.float32)
y = y/96.0
X = training_df.values[:, -1]
X = np.array(map(str2array, X))
X = X/255.0

In [8]:
X = X.reshape(-1, 1, 96, 96)

In [9]:
X.shape

(2140, 1, 96, 96)

In [10]:
out_channel = y.shape[1]

In [11]:
out_channel

30

In [12]:
class Net(nn.Module):
    def __init__(self, out_channel):
        super(Net, self).__init__()
        # (-1, 1, 96, 96)----->(-1, 16, 48, 48)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        # (-1, 16, 48, 48)----->(-1, 32, 24, 24)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        # (-1, 32, 24, 24)----->(-1, 64, 12, 12)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        # (-1, 64, 12, 12)----->(-1, 128, 6, 6)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        
        # (-1, 128, 6, 6)----->(-1, 128*6*6)----->(-1, 500)
        self.fc1 = nn.Linear(in_features=128*6*6, out_features=500, bias=True)
        self.fc2 = nn.Linear(in_features=500, out_features=out_channel, bias=True)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        
        x = x.view(-1, 128*6*6)
        x = self.fc1(x)
        x = self.fc2(x)
        out = x
        return out
net = Net(out_channel=out_channel)

In [13]:
X_tensor = torch.Tensor(torch.from_numpy(X))
y_tensor = torch.Tensor(torch.from_numpy(y))
traindataset = TensorDataset(X_tensor, y_tensor)
traindataloader = DataLoader(traindataset, batch_size=100, shuffle=True, num_workers=4)

In [14]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.0001, momentum=0, dampening=0, weight_decay=0, nesterov=False)

In [15]:
criterion = nn.MSELoss()

In [16]:
for epoch in range(0, 1, 1):
    loss_epoch = 0
    data_count = 0
    loss_avg_epoch = 0
    for i, (X_samples, y_samples) in enumerate(traindataloader):
        data_count = i
        optimizer.zero_grad()
#         print('X_samples.shape:', X_samples.shape)
#         print('y_samples.shape:', y_samples.shape)
        out_pred = net(X_samples)
#         print('out_pred.shape:', out_pred.shape)
        
        loss = criterion(out_pred, y_samples)
        loss.backward()
        
        loss_np = loss.data.numpy()
        loss_epoch += loss_np
        
        optimizer.step()
        break
    loss_avg_epoch = loss_epoch / (data_count * 1.0)
    print('epoch:{},loss_avg_epoch:{}'.format(epoch, loss_avg_epoch))

epoch:0,loss_avg_epoch:inf


In [17]:
# torch.save(net.state_dict(), '../model/facial_keypoints_model.pt')